# pixelSNAIL (refactored) for TensorFlow 1.0

Refactoring:

- Remove multiple GPU code
- Move helper functions to top
- Removed MPI influence
- Remove Queue Runners
- Use tf.Session() rather than one from utils
- Remove tf_utils
- Cleaned up training loop to use ImageGenerator from keras

In [1]:
import os
import sys
import time
import json
import numpy as np

import tensorflow as tf

import argparse
import datetime
import dateutil.tz
import functools

import nn as nn
import model as pxpp_models
import loaders
import plotting

In [2]:
from tensorflow.python.ops import variables
def passthrough(obj, value): 
    return value
variables.Variable._build_initializer_expr=passthrough

In [3]:
parser = argparse.ArgumentParser()

# data I/O
parser.add_argument('-i', '--data_dir', type=str, default='./data',
                    help='Location for the dataset')
parser.add_argument('-o', '--save_dir', type=str, default='./data/save',
                    help='Location for parameter checkpoints and samples')
parser.add_argument('-d', '--data_set', type=str, default='cifar',
                    help='Can be either cifar|imagenet')
parser.add_argument('-t', '--save_interval', type=int, default=50,
                    help='Every how many epochs to write checkpoint/samples?')
parser.add_argument('-r', '--load_params', type=str,
                    help='Restore training from previous model checkpoint?')

# model
parser.add_argument('--model', type=str, default="model_spec",
                    help='name of the model')
parser.add_argument('-q', '--nr_resnet', type=int, default=1,
                    help='Number of residual blocks per stage of the model')
parser.add_argument('-n', '--nr_filters', type=int, default=64,
                    help='Number of filters to use across the model. Higher = larger model.')
parser.add_argument('-m', '--nr_logistic_mix', type=int, default=10,
                    help='Number of logistic components in the mixture. Higher = more flexible model')
parser.add_argument('-z', '--resnet_nonlinearity', type=str, default='elu',
                    help='Which nonlinearity to use in the ResNet layers. One of "concat_elu", "elu", "relu" ')
parser.add_argument('-c', '--class_conditional', dest='class_conditional',
                    action='store_false', help='Condition generative model on labels?')
# optimization
parser.add_argument('-l', '--learning_rate', type=float,
                    default=1e-3, help='Base learning rate')
parser.add_argument('-e', '--lr_decay', type=float, default=0.999998,
                    help='Learning rate decay, applied every step of the optimization')
parser.add_argument('-b', '--batch_size', type=int, default=32,
                    help='Batch size during training per GPU')
parser.add_argument('-a', '--init_batch_size', type=int, default=32,
                    help='How much data to use for data-dependent initialization.')
parser.add_argument('-p', '--dropout_p', type=float, default=0.5,
                    help='Dropout strength (i.e. 1 - keep_prob). 0 = No dropout, higher = more dropout.')
parser.add_argument('-x', '--max_epochs', type=int, default=5000,
                    help='How many epochs to run in total?')
parser.add_argument('-g', '--nr_gpu', type=int, default=None,
                    help='How many GPUs to distribute the training across? Defaults to all.')

# evaluation
parser.add_argument('--polyak_decay', type=float, default=0.9995,
                    help='Exponential decay rate of the sum of previous model iterates during Polyak averaging')
# reproducibility
parser.add_argument('-s', '--seed', type=int, default=42,
                    help='Random seed to use')
args = parser.parse_args("")

args.class_conditional = False  
args.nr_gpu            = 1
args.batch_size        = 32
args.init_batch_size   = 32
args.dropout           = 0.2
args.learning_rate     = 3e-3
args.nr_resnet         = 2
args.nr_filters        = 128
args.nr_logistic_mix   = 10

# fix random seed for reproducibility
rng = np.random.RandomState(args.seed)
tf.set_random_seed(args.seed)

In [4]:
def lprint(*a, **kw):
    print(*a, **kw)
    print(*a, **kw, file=f_log)

def _get_batch(is_training):
    if is_training:
        x = train_data.__next__()
    else:
        x = test_data.__next__()
    x = np.cast[np.float32]((x - 127.5) / 127.5)
    return dict(x=x)


def sample_from_model(sess, n_samples=args.nr_gpu * args.batch_size, epoch=0):
    sample_x = np.zeros((0,) + obs_shape, dtype=np.float32)
    while len(sample_x) < n_samples:
        x_gen = np.zeros((args.batch_size,) + obs_shape, dtype=np.float32)

        for yi in range(obs_shape[0]):
            for xi in range(obs_shape[1]):
                new_x_gen_np = sess.run(new_x_gen, {xs: x_gen})
                x_gen[:, yi, xi, :] = new_x_gen_np[:, yi, xi, :]

        sample_x = np.concatenate((sample_x, x_gen), axis=0)

    img_tile = plotting.img_tile(sample_x[:int(np.floor(np.sqrt(n_samples)) ** 2)],
                                 aspect_ratio=1.0,
                                 border_color=1.0,
                                 stretch=True)
    img = plotting.plot_img(img_tile, title=args.data_set + ' samples, epoch ' +str(epoch))
    plotting.plt.savefig(
        os.path.join(save_dir, '{}s_samples_{}.png'.format(args.data_set, epoch)))
    #     np.save(os.path.join(save_dir, '%s_samples.npy' % args.data_set), sample_x)
    plotting.plt.close('all')


# turn numpy inputs into feed_dict for use with tensorflow
def make_feed_dict(data, init=False):
    if type(data) is tuple:
        x, y = data
    else:
        x = data
        y = None
    # input to pixelCNN is scaled from uint8 [0,255] to float in range [-1,1]
    x = np.cast[np.float32]((x - 127.5) / 127.5)
    if init:
        feed_dict = {x_init: x}
    else:
        feed_dict = {xs: x}
        if y is not None:
            feed_dict.update({ys: y})
    return feed_dict

In [5]:
# Setup file/folder structure

timestamp = datetime.datetime.now(dateutil.tz.tzlocal()).strftime('%Y_%m_%d_%H_%M_%S')
logdir = 'pixelsnail_%s_%s' % (args.data_set, timestamp)

args.save_dir = os.path.join(args.save_dir, logdir)
save_dir      = args.save_dir

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    f_log = open(os.path.join(save_dir, 'print.log'), 'w')

lprint('input args:\n', json.dumps(vars(args), indent=4,separators=(',', ':'))) 

input args:
 {
    "data_dir":"./data",
    "save_dir":"./data/save/pixelsnail_cifar_2020_05_07_17_13_39",
    "data_set":"cifar",
    "save_interval":50,
    "load_params":null,
    "model":"model_spec",
    "nr_resnet":2,
    "nr_filters":128,
    "nr_logistic_mix":10,
    "resnet_nonlinearity":"elu",
    "class_conditional":false,
    "learning_rate":0.003,
    "lr_decay":0.999998,
    "batch_size":32,
    "init_batch_size":32,
    "dropout_p":0.5,
    "max_epochs":5000,
    "nr_gpu":1,
    "polyak_decay":0.9995,
    "seed":42,
    "dropout":0.2
}


In [6]:
# load training data
train_data, test_data = loaders.load_cifar_data(None, batch_size=args.batch_size) # loaders.load_gemstone_data(None, batch_size=args.batch_size) #
obs_shape = (32,32,3)

In [7]:
# The tensorflow session
sess = tf.Session()

In [8]:
# data place holders
x_init = tf.placeholder(tf.float32,shape=(args.init_batch_size,) + obs_shape)
xs     = tf.placeholder(tf.float32, shape=(args.batch_size,) + obs_shape)
# if the model is class-conditional we'll set up label placeholders + one-hot encodings 'h' to condition on
h_init   = None
h_sample = None
hs       = h_sample

In [9]:
# set up the model
model_opt = {'nr_resnet'          : args.nr_resnet, 
             'attention'          : True,
             'nr_attn_block'      : 2,
             'nr_filters'         : args.nr_filters,
             'nr_logistic_mix'    : args.nr_logistic_mix, 
             'resnet_nonlinearity': args.resnet_nonlinearity}

lprint('model args:\n'+json.dumps(model_opt, indent=4,separators=(',', ':'))) 
# run once for data dependent initialization of parameters
model = tf.make_template('model', getattr(pxpp_models, args.model + ""))

model args:
{
    "nr_resnet":2,
    "attention":true,
    "nr_attn_block":2,
    "nr_filters":128,
    "nr_logistic_mix":10,
    "resnet_nonlinearity":"elu"
}


In [10]:
import time

In [11]:
gen_par = model(x_init, h_init, init=True,dropout_p=args.dropout_p, **model_opt)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# Number of parameters in the model
all_params = tf.trainable_variables()
lprint('# of Parameters', sum(np.prod(p.get_shape().as_list()) for p in all_params))

# of Parameters 2520890


In [13]:
# keep track of moving average
ema = tf.train.ExponentialMovingAverage(decay=args.polyak_decay)
maintain_averages_op = tf.group(ema.apply(all_params))

# Develop the train/test loss
loss_gen, loss_gen_test, grads = [], [], []
  
gen_par = model(xs, hs, ema=None,dropout_p=args.dropout_p, **model_opt)
if isinstance(gen_par, tuple) and len(gen_par) == 3:
    loss_gen.append(nn.discretized_mix_logistic_loss_per_chn(xs, *gen_par))
else:
    loss_gen.append(nn.discretized_mix_logistic_loss(xs, gen_par))
grads.append(tf.gradients(loss_gen, all_params))

gen_par = model(xs, hs, ema=ema, dropout_p=0., **model_opt)
if isinstance(gen_par, tuple) and len(gen_par) == 3:
    loss_gen_test.append(nn.discretized_mix_logistic_loss_per_chn(xs, *gen_par))
else:
    loss_gen_test.append(nn.discretized_mix_logistic_loss(xs, gen_par))

# add losses and gradients together and get training updates
tf_lr = tf.placeholder(tf.float32, shape=[])

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
# training op
optimizer = tf.group(nn.adam_updates(all_params, 
                                     grads[0], 
                                     lr=tf_lr, 
                                     mom1=0.95, 
                                     mom2=0.9995, 
                                     eps=1e-6), maintain_averages_op)

# convert loss to bits/dim

norm_const = np.log(2.) * np.prod(obs_shape) * args.batch_size

bits_per_dim      = loss_gen[0] / norm_const
bits_per_dim_test = loss_gen_test[0] / norm_const

bits_per_dim      = tf.check_numerics(bits_per_dim, 'train loss is nan')
bits_per_dim_test = tf.check_numerics(bits_per_dim_test, 'test loss is nan')

In [15]:
# for sampling the model
gen_par = model(xs, hs, ema=ema, dropout_p=0, **model_opt)
new_x_gen = nn.sample_from_discretized_mix_logistic(gen_par, args.nr_logistic_mix)

In [16]:
# init & save
saver = tf.train.Saver()

In [17]:
test_bpd = []
lr       = args.learning_rate

# manually retrieve exactly init_batch_size examples
feed_dict = make_feed_dict(train_data.next(), init=True)
# rewind the iterator back to 0 to do one full epoch
train_data.reset()  

In [18]:
# limit init batch to 1 to save memory
feed_dict[x_init] = feed_dict[x_init][:,:,:,:]

In [19]:
# t0 = time.time()
# initialize_all_variables(sess, feed_dict)
# t1 = time.time()
# print(t1-t0)

In [20]:
lprint('initializing the model...')

t0=time.time()
sess.run(tf.compat.v1.global_variables_initializer(), feed_dict)
t1 = time.time()
print(t1-t0)

args.load_params=True
if args.load_params:
    ckpt_file = "/home/tom/minerals/VQVAE/data/save/pixelsnail_cifar_2020_05_04_18_57_12/70/params_cifar.ckpt"
#     ckpt_file = save_dir + '/params_' + args.data_set + '.ckpt'
#     ckpt_file = args.load_params
    lprint('restoring parameters from', ckpt_file)
    saver.restore(sess, ckpt_file)

initializing the model...
1.6069750785827637
restoring parameters from /home/tom/minerals/VQVAE/data/save/pixelsnail_cifar_2020_05_04_18_57_12/70/params_cifar.ckpt
INFO:tensorflow:Restoring parameters from /home/tom/minerals/VQVAE/data/save/pixelsnail_cifar_2020_05_04_18_57_12/70/params_cifar.ckpt


In [21]:
#
# Assign values to all of the trainable parameters, which came from the initialization
#

my_vals = sess.run(all_params)
vals    = [np.zeros_like(v) for v in my_vals]
vals    = my_vals

assign_ops = [var.assign(val) for var, val in zip(all_params, vals)]

In [22]:
# This should give non-zero values
sess.run(assign_ops)

[array([[[[ 0.03125848, -0.11825117,  0.04057691, ..., -0.01712547,
            0.01151167, -0.04319537],
          [-0.06934398,  0.09202808,  0.01834022, ..., -0.03696219,
            0.05593271, -0.03781819],
          [-0.07081138,  0.15901692, -0.05976655, ...,  0.06186312,
            0.02830363, -0.05621216],
          [ 0.06981034, -0.15115441,  0.08224552, ..., -0.00128862,
            0.01708993, -0.05083424]],
 
         [[ 0.04029913, -0.05438673,  0.00635918, ...,  0.0365679 ,
           -0.03026514,  0.02324168],
          [ 0.05095311,  0.16427185, -0.07589711, ..., -0.00532264,
            0.08524626,  0.05388592],
          [-0.12753354,  0.01431052,  0.02633061, ..., -0.05516662,
           -0.04835953, -0.00333179],
          [-0.00861505, -0.14454529,  0.02099326, ..., -0.02595226,
            0.02887019, -0.03613016]],
 
         [[-0.11346294, -0.11771443, -0.03161078, ...,  0.05878973,
            0.09758575,  0.15377922],
          [-0.00217147,  0.00746953, -0.

In [23]:
lprint('starting training')

print(save_dir)
training_file_path = save_dir +os.sep + "training.txt"
if not os.path.exists(training_file_path):
    with open(training_file_path, "w") as f:
        f.write("")
testing_file_path = save_dir +os.sep + "testing.txt"
if not os.path.exists(testing_file_path):
    with open(testing_file_path, "w") as f:
        f.write("")


for epoch in range(args.max_epochs):
    begin = time.time()
    
    
    # train for one epoch
    train_losses = []
    for i, d in enumerate(train_data):
        print("{}/{}".format(i,len(train_data)), end="\r")
        if i == len(train_data):
            break
        feed_dict = make_feed_dict(d)
        lr *= args.lr_decay
        feed_dict.update({ tf_lr: lr })
        
        # forward + backward pass
        l, _ = sess.run([bits_per_dim, optimizer], feed_dict)
        train_losses.append(l)

    with open(training_file_path, "a") as f:
        f.write("{}".format(epoch)+",")
        f.write(",".join([str(t) for t in train_losses]))
        f.write("\n")

    train_loss_gen = np.mean(train_losses)

    # compute likelihood over test data
    test_losses = []
    for i, d in enumerate(test_data):
        print("{}/{}".format(i,len(test_data)), end="\r")
        if i == len(test_data):
            break
            
        feed_dict = make_feed_dict(d)
        l = sess.run(bits_per_dim_test, feed_dict)
        
        test_losses.append(l)
        
    with open(testing_file_path, "a") as f:
        f.write("{}".format(epoch)+",")
        f.write(",".join([str(t) for t in test_losses]))
        f.write("\n")
            
    test_loss_gen = np.mean(test_losses)

    # log progress to console
    stats = dict(epoch=epoch, 
                 time=time.time() - begin, 
                 lr=lr,
                 train_bpd=train_loss_gen,
                 test_bpd=test_loss_gen)

    lprint('-' * 16)
    for k, v in stats.items():
        lprint('%s:\t%s' % (k, v))
    if epoch % 2 == 0:
        path = os.path.join(save_dir, str(epoch))
        os.makedirs(path, exist_ok=True)
        saver.save(sess, os.path.join(path, 'params_%s.ckpt' % args.data_set))

    if epoch%2 == 0:
        sample_from_model(sess, epoch=epoch)

starting training
./data/save/pixelsnail_cifar_2020_05_07_17_13_39
----------------
epoch:	0
time:	255.94497108459473
lr:	0.002990642614489222
train_bpd:	3.374213
test_bpd:	3.2620556
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'adam_t' has type str, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'adam_t' has type str, but expected one of: int, long, bool
----------------
epoch:	1
time:	251.2241177558899
lr:	0.0029813144158663085
train_bpd:	3.3692691
test_bpd:	3.2612092
----------------
epoch:	2
time:	250.8839614391327
lr:	0.0029720153130936154
train_bpd:	3.369402
test_bpd:	3.2609744
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'adam_t' has type str, but expected one of: 

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: train loss is nan : Tensor had NaN values
	 [[node CheckNumerics (defined at /home/tom/anaconda3/envs/tf15_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[CheckNumerics/_1519]]
  (1) Invalid argument: train loss is nan : Tensor had NaN values
	 [[node CheckNumerics (defined at /home/tom/anaconda3/envs/tf15_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'CheckNumerics':
  File "/home/tom/anaconda3/envs/tf15_gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tom/anaconda3/envs/tf15_gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tom/anaconda3/envs/tf15_gpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tom/anaco